In [ ]:
from src import NeuralTester, Config
import torch
from src.learner import RevDELearner
from src.objective_functions import get_penalized_distance
from models import load_stylegan
import numpy as np
import logging
import wandb
import sys

## Example usage of the Neural Tester
We use wandb to log progress in various steps!

In [ ]:
wandb.login()

In [ ]:
# Setup logging to be able to view logging outputs in notebooks.
logging.basicConfig(level=logging.INFO, format='[%(asctime)s - %(name)s - %(levelname)s] %(message)s')
logger = logging.getLogger()
logger.handlers[0].stream = sys.stdout

In [ ]:
# Define the configurations for our experiments.
learner_params = {
    "f": 0.9,
    "cr": 0.5,
    "bounds": (0,1),
    "continuous": True,
    "population_size": 30,
}

conf = Config(
    samples_per_class=10,
    generations=50,
    mix_dim_range=(0,8),
    predictor="../models/wrn_mnist.pkl",
    generator="../models/sg2_mnist.pkl",
    learner=RevDELearner,
    learner_params=learner_params
)

In [ ]:
predictor = torch.load(conf.predictor)  # The System under test (SUT)
generator = load_stylegan(conf.generator)  # The generator network (a stylegan in this case)
learner = conf.learner(
    x0=np.random.rand(learner_params["population_size"], conf.genome_size),
    **learner_params
)  # The learner for search based optimization of candidates.
objective_function = get_penalized_distance  # The objective function to calculate fitness with.
device = torch.device("cuda")  # The target device for all operaitons.

In [ ]:
tester = NeuralTester(
    predictor=predictor,
    generator=generator,
    learner=learner,
    objective_function=objective_function,
    num_generations=conf.generations,
    mix_dim_range=conf.mix_dim_range,
    device=device,
)  # Here we initialize the Tester object.

In [ ]:
tester.test(num_classes=10, samples_per_class=conf.samples_per_class)  # We start the testing procedure.